# Text Binary Classification (scikit-learn)

In this __Machine Learing Snippet__ we use scikit-learn (http://scikit-learn.org/) and ebooks from Project Gutenberg (https://www.gutenberg.org/) to create text binary classifier, which can classify German and English text.

For our snippet we use the following ebooks:
- Alice's Adventures in Wonderland by Lewis Carroll (English), https://www.gutenberg.org/ebooks/28885
- Alice's Abenteuer im Wunderland by Lewis Carroll (German), https://www.gutenberg.org/ebooks/19778

__Note:__
The eBooks are for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org

### Data Preparation

Prepare the English and German text.


In [51]:
import re

txt_german = open('data/pg19778.txt', 'r').read()
txt_english = open('data/pg28885.txt', 'r').read()

feat_german = txt_german[5000: len(txt_german) - 20000].lower().strip().split()
feat_english = txt_english[5000: len(txt_english) - 20000].lower().strip().split()

def remove_special_chars(x):
    
    chars = ['_', '(', ')', '*', '"', '[', ']', '?', '!', ',', '.', '»', '«', ':', ';']
    for c in chars:
        x = x.replace(c, '')
    
    # remove numbers
    x = re.sub('\d', '', x)
    
    return x

feat_english = [remove_special_chars(x) for x in feat_english]
feat_german = [remove_special_chars(x) for x in feat_german]

print('tokens (german)', len(feat_german))
print('tokens (english)', len(feat_english))


tokens (german) 24934
tokens (english) 26678


### Feature Extraction
Try to cut off the header and footer of the ebook. We use fixed values, this is not precise but will do the job.

In [53]:

def create_text_sample(x):
    data = []
    text = []
    for i, f in enumerate(x):
        text.append(f)
        if i % 200 == 0 and i != 0:
            data.append(' '.join(text))
            text = []
    return data
    

print('len', len(data))


print('german', len(feat_german))
print('english', len(feat_english))


print(create_text_sample(feat_german)[3])


len 133
german 24934
english 26678
nun dinah sage die wahrheit hast du je einen spatzen gefressen da mit einem male plump plump kam sie auf einen haufen trocknes laub und reisig zu liegen -- und der fall war aus alice hatte sich gar nicht weh gethan sie sprang sogleich auf und sah in die höhe aber es war dunkel über ihr vor ihr lag ein zweiter langer gang und sie konnte noch eben das weiße kaninchen darin entlang laufen sehen es war kein augenblick zu verlieren fort rannte alice wie der wind und hörte es gerade noch sagen als es um eine ecke bog o ohren und schnurrbart wie spät es ist sie war dicht hinter ihm aber als sie um die ecke bog da war das kaninchen nicht mehr zu sehen sie befand sich in einem langen niedrigen corridor der durch eine reihe lampen erleuchtet war die von der decke herabhingen zu beiden seiten des corridors waren thüren aber sie waren alle verschlossen alice versuchte jede thür erst auf einer seite dann auf der andern endlich ging sie traurig in der mitte entlang

### Modeling

### Evaluation